# EXONS Cas9 vs Cpf1 Chopchop


In [1]:
import sys; print(sys.version)
import os
import glob
import subprocess
import multiprocessing
import io
from collections import OrderedDict
import json
import pickle

import numpy as np; print('numpy', np.__version__)
import scipy; print('scipy', scipy.__version__)
import pandas as pd; print('pandas',pd.__version__)
import allel; print('scikit-allel', allel.__version__)
import zarr; print('zarr', zarr.__version__)

import matplotlib as mpl
import matplotlib 

import statsmodels; print('statsmodels', statsmodels.__version__)
import statsmodels.api as sm

from IPython.display import display, HTML

3.11.5 | packaged by conda-forge | (main, Aug 27 2023, 03:34:09) [GCC 12.3.0]
numpy 1.26.0
scipy 1.12.0
pandas 2.1.1
scikit-allel 1.3.7
zarr 2.16.1
statsmodels 0.14.1


In [2]:
%matplotlib inline
mpl.rcParams['figure.facecolor'] = '#BBBBBB'

In [3]:
# bit of code to showing all a dataframe
from IPython.display import display, HTML
def display_all(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', None):
        display(df)
def display_all_cols(df, max_rows=60):
    with pd.option_context('display.max_rows', max_rows, 'display.max_columns', None, 'display.width', None):
        display(df)

## Settings/Constants

In [4]:
SETNAME = 'AAEL_Af'

if SETNAME == 'AgamGF':
    ## AgamGF ##########
    GENOME = "AgamP4.12"
    GENE_PREFIX = "AGAP"
    SAMPLE_LIST_FN = None
    CHROMS = ['2R','2L','3R','3L','X']
    GFFFN = "data/Anopheles-gambiae-PEST_BASEFEATURES_AgamP4.12.gff3"
    ZVCF = 'data/YLAgamGF+Ag1000G_pfilt.vcf.gz.zarr'
    CHOPCHOP_OUT_DIR = {'Cas9':"Cas9_all_transcripts_run",
                        'Cpf1':"Cpf1_all_transcripts_run"}
    TRANSFN = "data/AgamP4.12.transcript_list"

elif SETNAME == 'AAEL':
    ## AaegL5 #########
    GENOME = "AaegL5.1"
    GENE_PREFIX = "AAEL"
    SAMPLE_LIST_FN = None
    CHROMS = ['1','2','3']
    GFFFN = "data/Aedes-aegypti-LVP_AGWG_BASEFEATURES_AaegL5.1.gff3"
    ZVCF = 'data/AaegL5_2023-03.vcf.gz.zarr'
    CHOPCHOP_OUT_DIR = {'Cas9':"AaegL5_Cas9_all_transcripts_run",
                        'Cpf1':"AaegL5_Cpf1_all_transcripts_run"}
    TRANSFN = "data/Aedes-aegypti-LVP_AGWG_BASEFEATURES_AaegL5.1.transcript_list"

elif SETNAME == 'AAEL_Am':
    ## AaegL5 American samples #########
    GENOME = "AaegL5.1"
    GENE_PREFIX = "AAEL"
    SAMPLE_LIST_FN = "AAEG_sample_group_Am.csv"
    CHROMS = ['1','2','3']
    GFFFN = "data/Aedes-aegypti-LVP_AGWG_BASEFEATURES_AaegL5.1.gff3"
    ZVCF = 'data/AaegL5_2023-03.vcf.gz.zarr'
    CHOPCHOP_OUT_DIR = {'Cas9':"AaegL5_Cas9_all_transcripts_run",
                        'Cpf1':"AaegL5_Cpf1_all_transcripts_run"}
    TRANSFN = "data/Aedes-aegypti-LVP_AGWG_BASEFEATURES_AaegL5.1.transcript_list"
    
elif SETNAME == 'AAEL_Af':
    ## AaegL5 American samples #########
    GENOME = "AaegL5.1"
    GENE_PREFIX = "AAEL"
    SAMPLE_LIST_FN = "AAEG_sample_group_Af.csv"
    CHROMS = ['1','2','3']
    GFFFN = "data/Aedes-aegypti-LVP_AGWG_BASEFEATURES_AaegL5.1.gff3"
    ZVCF = 'data/AaegL5_2023-03.vcf.gz.zarr'
    CHOPCHOP_OUT_DIR = {'Cas9':"AaegL5_Cas9_all_transcripts_run",
                        'Cpf1':"AaegL5_Cpf1_all_transcripts_run"}
    TRANSFN = "data/Aedes-aegypti-LVP_AGWG_BASEFEATURES_AaegL5.1.transcript_list"
    
else:
    assert False, f"UNKNOWN SETNAME '{SETNAME}'"
    
    
##############################################################

MIN_GC = 40 # minimum allowable GC percentage (inclusive)
MAX_GC = 80 # maximum allowable GC percentage (inclusive)
MIN_EFFICIENCY = 0.50 
MAX_OFFTARGET_HITS = [0,0,0,0] # maximum allowable total offtarget hits (MM0, MM1, MM2, MM3); None=don't filter

SUBSAMPLE_N = 0
IGNORE_ONE_CALL_VARIANTS = False

AWK_EXEC = '/usr/bin/awk'

###################################################################
# Cas9 gRNA: gRNA sequence (20 nt) in blue and PAM in red (N being any nucleotide)
# The cut site is 3 nucleotides in taking only the gRNA sequence as a reference or 6 nucleotides in if you count the 3 nucleotides for the PAM site (in red)
#
# 5’ GGCGTATATATCGCGTAGTC NGG 3’
# 5’ GGCGTATATATCGCGTA-(cut site)-GTC NGG 3’
#
# Cas12a gRNA: gRNA sequence (23 nt) in blue and PAM in red (N being any nucleotide)
# The cut site for the Cas12a is staggered. So, it makes two simultaneous cuts at two different positions (cut site 1 and cut site 2)
# Cut site one is 18–19 bases from the 3′ end of the PAM. The cut on the opposite strand of the DNA is 23 bases from the PAM, causing a 5′ overhang of 4–5 bases. 
#
# 5’ TTTN GGCGTATATATCGCGTAG-(cut site)-TCGGG 3’ – “cut site 1”
# 5’ TTTN GGCGTATATATCGCGTAGTCGGG-(cut site) 3’ – “cut site 2”
######
# Cas9 (gRNA is 23bp):
# 1 	34 	TTATCCTTGTTTCAAAC TATAGG 	2L:157348 	+  (loc+17)
#                             ^ 
# 3 	61 	TATTATGTATTAAATAA TATTGG 	2L:157466 	-  (loc+5)
#        rc CCAATA TTATTTAATACATAATA
#                ^
# Cpf1 (gRNA is 28bp):
# 3 	37 	TTTCAAACTATAGGTGCGTGAAA ATCG C 	2L:157357 	+ 	
#                                   ^    ^                (loc+23, loc+27)
# 1 	16 	TTTCACGCACCTATAGTTTGAAA CAAG G 	2L:157352 	- 	
#        rc C CTTG TTTCAAACTATAGGTGCGTGAAA
#           ^    ^                                        (loc+0, loc+4)
CUT_OFFSET_gRNApos_CDSpos = {'Cas9':17, 'Cpf1':23}
CUT_OFFSET_gRNApos_CDSneg = {'Cas9':17, 'Cpf1':27}
CUT_OFFSET_gRNAneg_CDSpos = {'Cas9':5,  'Cpf1':4}
CUT_OFFSET_gRNAneg_CDSneg = {'Cas9':5,  'Cpf1':0}


In [5]:
results_dict = OrderedDict()  # to collect results for final display

## Load the trascript list

In [6]:
translist = pd.read_csv(TRANSFN, header=None, names=['transcript_id'])
t = translist['transcript_id'].str.rsplit('-', n=1, expand=True)
t.columns = ['gene', 'splice_id']
translist = pd.concat((translist,t), axis=1)
translist

,transcript_id,gene,splice_id
0,AAEL008159-RA,AAEL008159,RA
1,AAEL026895-RA,AAEL026895,RA
2,AAEL020319-RA,AAEL020319,RA
3,AAEL025965-RB,AAEL025965,RB
4,AAEL025965-RA,AAEL025965,RA
...,...,...,...
29330,AAEL012106-RA,AAEL012106,RA
29331,AAEL012106-RH,AAEL012106,RH
29332,AAEL012106-RG,AAEL012106,RG
29333,AAEL012102-RA,AAEL012102,RA


In [7]:
#
# Get the table of CDSs from the gff3 file
#

t = translist.copy(deep=True)

assert t.shape[0] == t['transcript_id'].nunique(), "trascript list should have no duplicte transcript_ids"

t.set_index('transcript_id', inplace=True)

gffdf = pd.read_csv(GFFFN, sep='\t', comment='#', header=None,
            names=['seqid',
                   'source',
                   'type',
                   'start',
                   'end',
                   'score',
                   'strand',
                   'phase',
                   'attributes'],
                dtype={'seqid':str})

# total number of genes (just accounting / checking)
# note: assumes one entry per gene & pseudogene in gff
tmp = (gffdf['type'] == 'gene').sum()
print("total number of genes in gtf:", tmp)
results_dict['total genes in gtf'] = tmp
tmp2 = (gffdf['type'] == 'pseudogene').sum()
print("total number of pseudogenes in gtf:", tmp2)
# results_dict['total pseudogenes in gtf'] = tmp2
assert tmp+tmp2 == t['gene'].nunique(), "number of genes+pseudogenes in gtf should equal number of genes in gene_table"

# just look at CDS entries
cds = gffdf.loc[gffdf['type']=='CDS' ,:].copy(deep=True)
# add transcript_id to Exon entries
cds['transcript_id'] = cds['attributes'].apply(
    lambda x: [x3[1].strip("'") for x3 in [x2.split('=') for x2 in x.split(';')] if x3[0] == 'Parent' ][0])

# join with the transcript list to get the gene_ids
num_CDSs = cds.shape[0]
cds = cds.join(translist.set_index('transcript_id'), on='transcript_id', how='outer')

print(f"CDSs not matching a transcript from translist: {(cds['gene'].isna()).sum()}") # should be 0
assert cds['gene'].isna().sum()==0, "CDS not matching a transcript in translist"

print(f"translist entries without a CDS (presumably pseudogenes): {(cds['seqid'].isna()).sum()}")
# drop the non-matching translist entries
cds = cds.loc[~cds['seqid'].isna(),:]

# cast positions to ints
cds['start'] = cds['start'].astype(int)
cds['end'] = cds['end'].astype(int)

# Only take CDSs in the CRHOMS of interest
tmp = cds.shape[0]
cds = cds[cds['seqid'].isin(CHROMS)]
print(f'CDSs not in CHROMS {CHROMS}: {tmp-cds.shape[0]}')

cds.sort_values(['seqid','start','splice_id'], inplace=True)
cds.reset_index(drop=True, inplace=True)

cds

total number of genes in gtf: 14626
total number of pseudogenes in gtf: 382
CDSs not matching a transcript from translist: 0
translist entries without a CDS (presumably pseudogenes): 1019
CDSs not in CHROMS ['1', '2', '3']: 4067


,seqid,source,type,start,end,score,strand,phase,attributes,transcript_id,gene,splice_id
0,1,VectorBase,CDS,31666,31761,.,+,0,ID=AAEL012102-PA;Parent=AAEL012102-RA;protein_...,AAEL012102-RA,AAEL012102,RA
1,1,VectorBase,CDS,31666,31761,.,+,0,ID=AAEL012102-PB;Parent=AAEL012102-RB;protein_...,AAEL012102-RB,AAEL012102,RB
2,1,VectorBase,CDS,31832,32404,.,+,0,ID=AAEL012102-PA;Parent=AAEL012102-RA;protein_...,AAEL012102-RA,AAEL012102,RA
3,1,VectorBase,CDS,31832,32404,.,+,0,ID=AAEL012102-PB;Parent=AAEL012102-RB;protein_...,AAEL012102-RB,AAEL012102,RB
4,1,VectorBase,CDS,32495,32673,.,+,0,ID=AAEL012102-PA;Parent=AAEL012102-RA;protein_...,AAEL012102-RA,AAEL012102,RA
...,...,...,...,...,...,...,...,...,...,...,...,...
168730,3,VectorBase,CDS,409622594,409622821,.,-,0,ID=AAEL010311-PA;Parent=AAEL010311-RA;protein_...,AAEL010311-RA,AAEL010311,RA
168731,3,VectorBase,CDS,409622594,409622821,.,-,0,ID=AAEL010311-PB;Parent=AAEL010311-RB;protein_...,AAEL010311-RB,AAEL010311,RB
168732,3,VectorBase,CDS,409688525,409688690,.,+,0,ID=AAEL010318-PA;Parent=AAEL010318-RA;protein_...,AAEL010318-RA,AAEL010318,RA
168733,3,VectorBase,CDS,409695468,409696934,.,+,2,ID=AAEL010318-PA;Parent=AAEL010318-RA;protein_...,AAEL010318-RA,AAEL010318,RA


In [8]:
# drop CDSs which have length 1 (start==end)
print(f"DROPPING {(cds['start'] == cds['end']).sum()} CDSs where start==end")
cds = cds[cds['start'] != cds['end']].copy()


# look for overlapping CDS with the SAME GENE
print(f"number of CDS which are simple splice variants (same start,end,strand,gene): {cds.duplicated(['seqid','start','end','strand','gene'], keep='first').sum()}")

# drop simple splice variants
tmp = cds.shape[0]
cds.drop_duplicates(['seqid','start','end','strand','gene'], keep='first', inplace=True)
print(f"DROPPING {tmp-cds.shape[0]} simple spliace variants")



# overlap groups with same gene
overlap = ((cds['start']<=cds['end'].shift(1)) & (cds['seqid']==cds['seqid'].shift(1)) & (cds['gene']==cds['gene'].shift(1)))
cds['overlap_group'] = (~overlap).cumsum()
gb = cds.groupby('overlap_group', dropna=False)

assert (gb['gene'].nunique() == 1).all(), "all groups should contain only the same gene"

print(f"number of same-gene overlap groups with multiple CDSs: {(gb['gene'].count()>1).sum()}")
print(f"number of same-gene overlapping CDSs: {gb['gene'].count()[gb['gene'].count()>1].sum()}")

foo = gb.nth(0).copy()
foo['start'] = gb['start'].min()
foo['end'] = gb['end'].max()
foo['CDS cnt'] = gb.size()
foo['splice_id'] = gb['splice_id'].agg(lambda x: ','.join(x))

foo['full_len'] = gb['end'].max()-gb['start'].min()+1
foo['overlap_len'] = gb['end'].min()-gb['start'].max()+1

print(f"number of same-gene groups with CDSs of different lengths: {foo[foo['overlap_len'] != foo['full_len']].shape[0]}")

print("** number of protein coding genes:", cds['gene'].nunique())
results_dict['number of protein colding genes'] = cds['gene'].nunique()

cds_grps = foo
cds_grps

DROPPING 79 CDSs where start==end
number of CDS which are simple splice variants (same start,end,strand,gene): 103728
DROPPING 103728 simple spliace variants
number of same-gene overlap groups with multiple CDSs: 2667
number of same-gene overlapping CDSs: 5684
number of same-gene groups with CDSs of different lengths: 40203
** number of protein coding genes: 13601


,seqid,source,type,start,end,score,strand,phase,attributes,transcript_id,gene,splice_id,overlap_group,CDS cnt,full_len,overlap_len
0,1,VectorBase,CDS,NaN,NaN,.,+,0,ID=AAEL012102-PA;Parent=AAEL012102-RA;protein_...,AAEL012102-RA,AAEL012102,NaN,1,NaN,NaN,NaN
2,1,VectorBase,CDS,31832.0,32404.0,.,+,0,ID=AAEL012102-PA;Parent=AAEL012102-RA;protein_...,AAEL012102-RA,AAEL012102,RA,2,1.0,573.0,573.0
4,1,VectorBase,CDS,32732.0,33596.0,.,+,0,ID=AAEL012102-PA;Parent=AAEL012102-RA;protein_...,AAEL012102-RA,AAEL012102,RA,3,1.0,865.0,865.0
6,1,VectorBase,CDS,41767.0,41958.0,.,+,1,ID=AAEL012102-PA;Parent=AAEL012102-RA;protein_...,AAEL012102-RA,AAEL012102,RA,4,1.0,192.0,192.0
8,1,VectorBase,CDS,44349.0,44471.0,.,+,0,ID=AAEL012102-PA;Parent=AAEL012102-RA;protein_...,AAEL012102-RA,AAEL012102,RA,5,1.0,123.0,123.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168728,3,VectorBase,CDS,NaN,NaN,.,-,0,ID=AAEL010311-PA;Parent=AAEL010311-RA;protein_...,AAEL010311-RA,AAEL010311,NaN,61907,NaN,NaN,NaN
168730,3,VectorBase,CDS,NaN,NaN,.,-,0,ID=AAEL010311-PA;Parent=AAEL010311-RA;protein_...,AAEL010311-RA,AAEL010311,NaN,61908,NaN,NaN,NaN
168732,3,VectorBase,CDS,NaN,NaN,.,+,0,ID=AAEL010318-PA;Parent=AAEL010318-RA;protein_...,AAEL010318-RA,AAEL010318,NaN,61909,NaN,NaN,NaN
168733,3,VectorBase,CDS,NaN,NaN,.,+,2,ID=AAEL010318-PA;Parent=AAEL010318-RA;protein_...,AAEL010318-RA,AAEL010318,NaN,61910,NaN,NaN,NaN


In [9]:
# computing the cumulatilve length (bp) of the CDSs accounting for overlaps
overlap = ((cds['start']<=cds['end'].shift(1)) & (cds['seqid']==cds['seqid'].shift(1)))
cds['overlap_group_ignoring_gene'] = (~overlap).cumsum()

gb = cds.groupby('overlap_group_ignoring_gene', dropna=False)

print(f"number of overlap groups with multiple CDSs: {(gb['gene'].count()>1).sum()}")
print(f"number of overlapping CDSs: {gb['gene'].count()[gb['gene'].count()>1].sum()}")

total_coding_len = (gb['end'].max()-gb['start'].min()+1).sum()
print(f"** total coding length [bp]: {total_coding_len}")
results_dict['total coding length [bp]'] = total_coding_len

print(f"total coding len possibly multiple counting non-same-gene overlaps: {cds_grps['full_len'].sum()}")

number of overlap groups with multiple CDSs: 2681
number of overlapping CDSs: 5714
** total coding length [bp]: 23255031
total coding len possibly multiple counting non-same-gene overlaps: 8370802.0


In [10]:
# @TCC NOTE:  (JUST INFORMATIVE)
# The CDSs which overlap involving differen genes might be a problem if a chopchop target hits that overlap.
# We will check for that below by *ensuring each chopchop target hits only one gene*

# investigating overlaps between genes
overlap = ((cds_grps['start']<=cds_grps['end'].shift(1)) & (cds_grps['seqid'].shift(1)==cds_grps['seqid']))
cds_grps['overlap_group_ignoring_gene'] = (~overlap).cumsum()

gb = cds_grps.groupby('overlap_group_ignoring_gene', dropna=False)

print("number of overlaps involving diferent genes:", (gb['gene'].nunique() > 1).sum())
print("number of CDSs with overlaps involving different genes:", overlap.sum())
results_dict['overlapping CDSs with different gene ids'] = overlap.sum()

# print out all of the overlap groups with multiple genes
for i in gb.first()[gb['gene'].nunique() > 1].index.values:
    display(gb.get_group(i))
    print("full len:", gb.get_group(i)['end'].max()-gb.get_group(i)['start'].min()+1)
    print("overlap len:", gb.get_group(i)['end'].min()-gb.get_group(i)['start'].max()+1)


number of overlaps involving diferent genes: 10
number of CDSs with overlaps involving different genes: 37


,seqid,source,type,start,end,score,strand,phase,attributes,transcript_id,gene,splice_id,overlap_group,CDS cnt,full_len,overlap_len,overlap_group_ignoring_gene
306,1,VectorBase,CDS,5770858.0,5771183.0,.,-,0,ID=AAEL011377-PA;Parent=AAEL011377-RA;protein_...,AAEL011377-RA,AAEL011377,"RB,RA,RE",128,3.0,326.0,20.0,128
307,1,VectorBase,CDS,5771094.0,5771183.0,.,-,0,ID=AAEL011381-PA;Parent=AAEL011381-RA;protein_...,AAEL011381-RA,AAEL011381,RE,129,1.0,90.0,90.0,128


full len: 326.0
overlap len: 90.0


,seqid,source,type,start,end,score,strand,phase,attributes,transcript_id,gene,splice_id,overlap_group,CDS cnt,full_len,overlap_len,overlap_group_ignoring_gene
864,1,VectorBase,CDS,17935465.0,17936815.0,.,-,0,ID=AAEL000242-PD;Parent=AAEL000242-RD;protein_...,AAEL000242-RD,AAEL000242,"RA,RB",319,2.0,1351.0,1182.0,316
865,1,VectorBase,CDS,17936704.0,17936815.0,.,+,0,ID=AAEL000249-PA;Parent=AAEL000249-RA;protein_...,AAEL000249-RA,AAEL000249,RB,320,1.0,112.0,112.0,316


full len: 1351.0
overlap len: 112.0


...

full len: 537.0
overlap len: 105.0


,seqid,source,type,start,end,score,strand,phase,attributes,transcript_id,gene,splice_id,overlap_group,CDS cnt,full_len,overlap_len,overlap_group_ignoring_gene
49479,2,VectorBase,CDS,183898683.0,183900461.0,.,+,0,ID=AAEL022351-PA;Parent=AAEL022351-RA;protein_...,AAEL022351-RA,AAEL022351,"RB,RD",18239,2.0,1779.0,1328.0,18208
49480,2,VectorBase,CDS,183900098.0,183900461.0,.,+,0,ID=AAEL025226-PA;Parent=AAEL025226-RA;protein_...,AAEL025226-RA,AAEL025226,RD,18240,1.0,364.0,364.0,18208


full len: 1779.0
overlap len: 364.0


,seqid,source,type,start,end,score,strand,phase,attributes,transcript_id,gene,splice_id,overlap_group,CDS cnt,full_len,overlap_len,overlap_group_ignoring_gene
56717,2,VectorBase,CDS,312826244.0,312826682.0,.,+,0,ID=AAEL003602-PA;Parent=AAEL003602-RA;protein_...,AAEL003602-RA,AAEL003602,RA,21115,1.0,439.0,439.0,21081
56718,2,VectorBase,CDS,312826676.0,312827047.0,.,-,0,ID=AAEL003590-PA;Parent=AAEL003590-RA;protein_...,AAEL003590-RA,AAEL003590,RA,21116,1.0,372.0,372.0,21081


full len: 804.0
overlap len: 7.0


## Load and filter the chopchop targets

In [11]:
raw_targets = OrderedDict()
for mode, chopchop_out_dir in CHOPCHOP_OUT_DIR.items():
    print(mode, chopchop_out_dir)

    ### use awk to append the transcript_ids and merge all the chopchop outputs
    # stream than into pandas.read_csv
    cmd = (AWK_EXEC+" -F "+r"'\t' "
    "'BEGIN{OFS=FS}{if(FNR==1){if(NR==FNR){print $0, "+'"transcript_id"'+"}}else{print $0, FILENAME}}' "+
    "{}*".format(GENE_PREFIX))

    print(f"running in '{chopchop_out_dir}':\n{cmd}")
    with subprocess.Popen(cmd, cwd=chopchop_out_dir, shell=True, stdout=subprocess.PIPE) as p:
        d = pd.read_csv(p.stdout, sep='\t', index_col=False)

    print(f"{d.shape[0]} raw targets loaded")

    # add a gene_id column
    d['gene_id'] = d['transcript_id'].apply(lambda x: x.rsplit(sep='-', maxsplit=1)[0])

    ### drop duplicate targets
    # chopchop duplicates some targets for unknown reasons... just exclude these
    d = d.drop_duplicates(subset=['Target sequence', 'Genomic location','gene_id'], keep='first').sort_values('gene_id')
    print(f"{d.shape[0]} different targets (sequence, location, gene_id)")
    
    raw_targets[mode] = d.copy()

Cas9 AaegL5_Cas9_all_transcripts_run
running in 'AaegL5_Cas9_all_transcripts_run':
/usr/bin/awk -F '\t' 'BEGIN{OFS=FS}{if(FNR==1){if(NR==FNR){print $0, "transcript_id"}}else{print $0, FILENAME}}' AAEL*
10116163 raw targets loaded
3861017 different targets (sequence, location, gene_id)
Cpf1 AaegL5_Cpf1_all_transcripts_run
running in 'AaegL5_Cpf1_all_transcripts_run':
/usr/bin/awk -F '\t' 'BEGIN{OFS=FS}{if(FNR==1){if(NR==FNR){print $0, "transcript_id"}}else{print $0, FILENAME}}' AAEL*
5725237 raw targets loaded
2243878 different targets (sequence, location, gene_id)


In [12]:
targets = OrderedDict()
for mode in raw_targets.keys():
    print("####", mode)

    d = raw_targets[mode].copy()

    # create chrom and pos columns from Genomic location
    d[['chrom','pos']] = d['Genomic location'].str.split(':',n=1,expand=True)
    d['pos'] = d['pos'].astype(int)
    d.sort_values(['chrom', 'pos'], inplace=True)

    # go ahead and just drop targets not hitting chroms of interest
    d = d[d['chrom'].isin(CHROMS)].copy()
    
    # add the locations of the cut-points to the table of targets
    d['cutpos'] = None
    d['cutneg'] = None
    d.loc[d['Strand']=='+','cutpos'] = d['pos'] + CUT_OFFSET_gRNApos_CDSpos[mode]
    d.loc[d['Strand']=='+','cutneg'] = d['pos'] + CUT_OFFSET_gRNApos_CDSneg[mode]
    d.loc[d['Strand']=='-','cutpos'] = d['pos'] + CUT_OFFSET_gRNAneg_CDSpos[mode]
    d.loc[d['Strand']=='-','cutneg'] = d['pos'] + CUT_OFFSET_gRNAneg_CDSneg[mode]
    assert d['cutpos'].isna().any() == False, "Error with determining positions"
    assert d['cutneg'].isna().any() == False, "Error with determining positions"
        

    ### Some targets can hit multiple overlapping genes
    # Just flag these for now...
    d['flag unique location'] = ~d.duplicated(subset=['Target sequence', 'Genomic location'], keep=False)
    print(f"{d['flag unique location'].sum()} targets at unique locations")
    print(f"{(d['flag unique location']==False).sum()} targets have non-unique locations")
    results_dict[f'{mode} targets with unique location'] = d['flag unique location'].sum()
    print(f"{len(d[~d['flag unique location']]['Genomic location'].unique())} locations hit by more than one target")
    print(f"{d['flag unique location'].sum()} ({100*d['flag unique location'].sum()/d.shape[0]:.2f}%) targets have unique locations")

    ### Each target needs to have a unique genomic location for the analysis to work
    # ensure there are no duplicate locations with the same gene_id (possibly from having variant transcripts from the same gene)
    assert d.loc[:,['Target sequence', 'Genomic location', 'gene_id']].duplicated().sum() == 0, 'Duplicate locations hitting the same gene_id... this is not handled properly so bailing'
    # remove targets without a unique location (hitting multiple genes) since they are non-specific
    d = d[d['flag unique location']==True].copy() 

    # NOTE: Below this point, targets should be unique locations hitting a single gene

    ### filters
    # MAX_OFFTARGET_HITS = [0,0,0,0] # examplr for on MM
    # create a numeric version of the MM3 column just taking the numbers from the '>=' results
    d['MM3_num'] = pd.to_numeric(d['MM3'], errors='coerce')
    d.loc[d['MM3_num'].isna(), 'MM3_num'] = pd.to_numeric(d.loc[d['MM3_num'].isna(),'MM3'].str[2:], errors='coerce')
    d['sumMM'] = d.loc[:,['MM0','MM1','MM2','MM3_num']].sum(axis=1) # Not strictly needed, but good to keep track of accounting

    d['flt MM'] = True
    for i,col in enumerate(['MM0','MM1','MM2','MM3_num']):
        if MAX_OFFTARGET_HITS[i] is not None:
            d['flt MM'] = (d['flt MM']) & (d[col]<=MAX_OFFTARGET_HITS[i])
    print(f"{d['flt MM'].sum()} ({100*d['flt MM'].sum()/d.shape[0]:.2f}%) targets pass offtarget hits (mismatch) filters")

    ### GC filter
    d['flt GC'] = ((d["GC content (%)"] >= MIN_GC) & (d["GC content (%)"] <= MAX_GC))
    print(f"{d['flt GC'].sum()} ({100*d['flt GC'].sum()/d.shape[0]:.2f}%) targets pass GC filters")

    ### Efficiency filter
    if mode == 'Cas9':
        min_efficiency = MIN_EFFICIENCY
    elif mode == 'Cpf1':
        min_efficiency = MIN_EFFICIENCY*100
    else:
        raise ValueError
    d['flt Eff'] = d["Efficiency"] >= min_efficiency
    print(f"{d['flt Eff'].sum()} ({100*d['flt Eff'].sum()/d.shape[0]:.2f}%) targets pass Efficiency filters")

    d.sort_values(['chrom','pos'], inplace=True)
    d.reset_index(drop=True, inplace=True)
    d['idx'] = d.index

    results_dict[f'{mode} targets pass MM flt'] = d['flt MM'].sum()
    results_dict[f'{mode} targets pass GC flt'] = d['flt GC'].sum()
    results_dict[f'{mode} targets pass Eff flt'] = d['flt Eff'].sum()
    
    targets[mode] = d.copy()

#### Cas9
3665000 targets at unique locations
22261 targets have non-unique locations
11121 locations hit by more than one target
3665000 (99.40%) targets have unique locations
858176 (23.42%) targets pass offtarget hits (mismatch) filters
3016380 (82.30%) targets pass GC filters
1946078 (53.10%) targets pass Efficiency filters
#### Cpf1
2126127 targets at unique locations
22833 targets have non-unique locations
11404 locations hit by more than one target
2126127 (98.94%) targets have unique locations
1695992 (79.77%) targets pass offtarget hits (mismatch) filters
1249707 (58.78%) targets pass GC filters
716463 (33.70%) targets pass Efficiency filters


In [13]:
results_dict

OrderedDict([('total genes in gtf', 14626),
             ('number of protein colding genes', 13601),
             ('total coding length [bp]', 23255031),
             ('overlapping CDSs with different gene ids', 37),
             ('Cas9 targets with unique location', 3665000),
             ('Cas9 targets pass MM flt', 858176),
             ('Cas9 targets pass GC flt', 3016380),
             ('Cas9 targets pass Eff flt', 1946078),
             ('Cpf1 targets with unique location', 2126127),
             ('Cpf1 targets pass MM flt', 1695992),
             ('Cpf1 targets pass GC flt', 1249707),
             ('Cpf1 targets pass Eff flt', 716463)])

In [14]:
d = targets['Cas9']
display_all_cols(d[(d['flt MM']==True) & (d['flt GC']==True) & (d['flt Eff']==True)])

,Rank,Target sequence,Genomic location,Strand,GC content (%),Self-complementarity,MM0,MM1,MM2,MM3,Efficiency,transcript_id,gene_id,chrom,pos,cutpos,cutneg,flag unique location,MM3_num,sumMM,flt MM,flt GC,flt Eff,idx
6,8,AATATACACACGTCGTTTCGAGG,1:31541,-,40,0,0,0,0,0,0.50,AAEL012102-RA,AAEL012102,1,31541,31546,31546,True,0.0,0.0,True,True,True,6
11,2,CGACAGCTGCCTTTGATCGGAGG,1:31726,+,60,0,0,0,0,0,0.69,AAEL012102-RA,AAEL012102,1,31726,31743,31743,True,0.0,0.0,True,True,True,11
20,5,AGCTCCTGTCCTTTTACTGGAGG,1:31874,-,50,0,0,0,0,0,0.61,AAEL012102-RA,AAEL012102,1,31874,31879,31879,True,0.0,0.0,True,True,True,20
34,7,TTTATGGAGAAGAGTGCACAGGG,1:32139,-,40,0,0,0,0,0,0.53,AAEL012102-RA,AAEL012102,1,32139,32144,32144,True,0.0,0.0,True,True,True,34
51,6,CAGGAATGTCGAACAAAGGTAGG,1:32387,+,45,0,0,0,0,0,0.59,AAEL012102-RA,AAEL012102,1,32387,32404,32404,True,0.0,0.0,True,True,True,51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3664908,20,GCGTCCTGTTGCTCCTCAGATGG,3:409696795,+,60,0,0,0,0,0,0.55,AAEL010318-RA,AAEL010318,3,409696795,409696812,409696812,True,0.0,0.0,True,True,True,3664908
3664942,10,GGAGCATAGCGAATCGCTGAAGG,3:409697160,+,55,2,0,0,0,0,0.61,AAEL010318-RA,AAEL010318,3,409697160,409697177,409697177,True,0.0,0.0,True,True,True,3664942
3664943,19,TAGCGAATCGCTGAAGGCTAAGG,3:409697166,+,50,3,0,0,0,0,0.58,AAEL010318-RA,AAEL010318,3,409697166,409697183,409697183,True,0.0,0.0,True,True,True,3664943
3664944,3,ATGAGCGTGCAGTACGGCAACGG,3:409697198,-,55,0,0,0,0,0,0.67,AAEL010318-RA,AAEL010318,3,409697198,409697203,409697203,True,0.0,0.0,True,True,True,3664944


In [15]:
# %%time
# mapping targets to CDSs

for mode in targets.keys():
    print('####', mode)
    
    d = targets[mode].copy() 

    d['gene_id'] = None

    ### Targets where cutpoint hits a CDS
    for chrom in CHROMS:
        
        for cds_strand,cut_col in [('+','cutpos'), ('-','cutneg')]: 
            print(chrom, cds_strand)
            cdstmp = cds.loc[(cds['seqid']==chrom) & (cds['strand']==cds_strand),:]       
            tmp = d.loc[d['chrom']==chrom].sort_values(cut_col)
            starts_idx = np.searchsorted(tmp[cut_col].values, cdstmp['start'].values, side='left')
            ends_idx = np.searchsorted(tmp[cut_col].values, cdstmp['end'].values, side='right')
        
     
            # some sanity testing
            cutpos_col_num = tmp.columns.get_loc(cut_col)
            for i in range(cdstmp.shape[0]):
                # if cdstmp.iloc[i]['end'] == tmp.iloc[ends_idx[i]-1,cutpos_col_num]:
                #     print(f"FOO: {cdstmp.iloc[i]['end']} == {tmp.iloc[ends_idx[i]-1,cutpos_col_num]}")
                #     display(cdstmp.iloc[i])
                #     display(tmp.iloc[ends_idx[i]-1,:])
                #     break

                if i>0 and cdstmp.iloc[i]['start'] < tmp.iloc[starts_idx[i]-1,cutpos_col_num]:
                    print("target in cds start missed")
                    break
                if cdstmp.iloc[i]['start'] > tmp.iloc[starts_idx[i],cutpos_col_num]:
                    print("first target is before cds start")
                    break
                if cdstmp.iloc[i]['end'] < tmp.iloc[ends_idx[i]-1,cutpos_col_num]:
                    print("end target is after cds end")
                    break
                if i<len(ends_idx)-1 and cdstmp.iloc[i]['end'] > tmp.iloc[ends_idx[i],cutpos_col_num]:
                    print("target in cds end missed")
                    break
            if i < len(ends_idx)-1:
                print(f"{tmp.iloc[starts_idx[i]-1,cutpos_col_num]} ({cds.iloc[i]['start']}) {tmp.iloc[starts_idx[i],cutpos_col_num]} ... {tmp.iloc[ends_idx[i]-1,cutpos_col_num]} ({cds.iloc[i]['end']}) {tmp.iloc[ends_idx[i],cutpos_col_num]}")
            else:
                print("all looks sane")

            # Assigning gene_id to each target
            tmp_idx_col_num = tmp.columns.get_loc('idx') # the 'idx' col maps values from tmp (slice of d) back to the index of d
            for i in range(cdstmp.shape[0]):
                if i%1000 == 0:
                    print(f"{i} of {cdstmp.shape[0]} : {i*100/cdstmp.shape[0]:2.2f}%")
                if starts_idx[i]<ends_idx[i]: 
                    gene_id = cdstmp.iloc[i]['gene']
                    # print(starts_idx[i], ends_idx[i], gene_id)
                    # display_all_cols(d.loc[tmp.iloc[starts_idx[i]:ends_idx[i],tmp_idx_col_num].values,:])
                    assert d.loc[tmp.iloc[starts_idx[i]:ends_idx[i],tmp_idx_col_num].values,'gene_id'].isin([None, gene_id]).all(), "problem: target appears to hit multiple genes"
                    d.loc[tmp.iloc[starts_idx[i]:ends_idx[i],tmp_idx_col_num].values,'gene_id'] = gene_id
                    # display_all_cols(d.loc[tmp.iloc[starts_idx[i]:ends_idx[i],tmp_idx_col_num].values,:])
        print(f"{d['gene_id'].nunique()-1} gene ids with a target in their CDSs")
        # more sanity checking
        assert (d.loc[:,('chrom','pos')].duplicated() == False).all(), "All targets should be unique locations"
       
        # save results
        targets[mode] = d.copy()

#### Cas9
1 +
all looks sane
0 of 7374 : 0.00%
1000 of 7374 : 13.56%
2000 of 7374 : 27.12%
3000 of 7374 : 40.68%
4000 of 7374 : 54.24%
5000 of 7374 : 67.81%
6000 of 7374 : 81.37%
7000 of 7374 : 94.93%
1 -
all looks sane
0 of 7614 : 0.00%
1000 of 7614 : 13.13%
2000 of 7614 : 26.27%
3000 of 7614 : 39.40%
4000 of 7614 : 52.53%
5000 of 7614 : 65.67%
6000 of 7614 : 78.80%
7000 of 7614 : 91.94%
3176 gene ids with a target in their CDSs
2 +
all looks sane
0 of 13332 : 0.00%
1000 of 13332 : 7.50%
2000 of 13332 : 15.00%
3000 of 13332 : 22.50%
4000 of 13332 : 30.00%
5000 of 13332 : 37.50%
6000 of 13332 : 45.00%
7000 of 13332 : 52.51%
8000 of 13332 : 60.01%
9000 of 13332 : 67.51%
10000 of 13332 : 75.01%
11000 of 13332 : 82.51%
12000 of 13332 : 90.01%
13000 of 13332 : 97.51%
2 -
all looks sane
0 of 13670 : 0.00%
1000 of 13670 : 7.32%
2000 of 13670 : 14.63%
3000 of 13670 : 21.95%
4000 of 13670 : 29.26%
5000 of 13670 : 36.58%
6000 of 13670 : 43.89%
7000 of 13670 : 51.21%
8000 of 13670 : 58.52%
9000 

In [16]:
display_all_cols(targets['Cas9'].iloc[153:173,:])

,Rank,Target sequence,Genomic location,Strand,GC content (%),Self-complementarity,MM0,MM1,MM2,MM3,Efficiency,transcript_id,gene_id,chrom,pos,cutpos,cutneg,flag unique location,MM3_num,sumMM,flt MM,flt GC,flt Eff,idx
153,102,ATAGCAAACAAGGACCAGCCTGG,1:41748,-,50,1,0,0,0,3,0.49,AAEL012106-RA,None,1,41748,41753,41753,True,3.0,3.0,False,True,False,153
154,86,CTGGTCCTTGTTTGCTATTGTGG,1:41753,+,45,0,0,0,0,2,0.37,AAEL012106-RA,AAEL012106,1,41753,41770,41770,True,2.0,2.0,False,True,False,154
155,9,GTCCTTGTTTGCTATTGTGGTGG,1:41756,+,45,0,0,0,0,0,0.59,AAEL012106-RA,AAEL012106,1,41756,41773,41773,True,0.0,0.0,True,True,True,155
156,49,GTCCACCACAATAGCAAACAAGG,1:41758,-,45,0,0,0,0,1,0.50,AAEL012106-RA,None,1,41758,41763,41763,True,1.0,1.0,False,True,True,156
157,12,TTTGCTATTGTGGTGGACACCGG,1:41763,+,45,0,0,0,0,0,0.52,AAEL012106-RA,AAEL012106,1,41763,41780,41780,True,0.0,0.0,True,True,True,157
158,32,TAGCTACTTCAACGGTAGTCCGG,1:41782,-,45,1,0,0,0,0,0.37,AAEL012106-RA,AAEL012106,1,41782,41787,41787,True,0.0,0.0,True,True,False,158
159,139,CGATGAATTAGCTACTTCAACGG,1:41790,-,35,0,0,0,3,2,0.49,AAEL012106-RA,AAEL012106,1,41790,41795,41795,True,2.0,5.0,False,False,False,159
160,157,TCATCGAATGTTTTATCGATAGG,1:41807,+,30,2,0,0,1,7,0.47,AAEL012106-RA,AAEL012106,1,41807,41824,41824,True,7.0,8.0,False,False,False,160
161,70,ATTCGATCGTCATACAACGTTGG,1:41831,+,40,0,0,0,0,2,0.55,AAEL012106-RA,AAEL012106,1,41831,41848,41848,True,2.0,2.0,False,True,True,161
162,15,GCGGTACTGGCTGGAACTGCTGG,1:41870,-,65,0,0,0,0,0,0.50,AAEL012106-RA,AAEL012106,1,41870,41875,41875,True,0.0,0.0,True,True,True,162


In [17]:
# make a flt column for if it hits a CDS
for mode in targets.keys():
    targets[mode]['flt hitsCDS'] = False
    targets[mode].loc[targets[mode]['gene_id'].notna(),'flt hitsCDS'] = True

In [18]:
# filter for targets hitting a CDS (have a gene_id) and apply previous filters to get potential_targets

potential_targets = OrderedDict()

for mode in targets.keys():
    print('####', mode)
    d = targets[mode]
    potential_targets[mode] = d[(d['flt MM']==True) & (d['flt GC']==True) & (d['flt Eff']==True) & (d['gene_id'].notna())].copy()

    tmp = d.loc[d['gene_id'].notna(),:].shape[0]    
    results_dict[f'{mode} targets hit CDS'] = tmp
    print(f"{tmp} ({100*tmp/d.shape[0]:.2g}%) targets hit a CDS")
        
    results_dict[f'{mode} targets pass all fileters'] = potential_targets[mode].shape[0]
    print(f"targets pass all fileters: {results_dict[f'{mode} targets pass all fileters']} ({100*results_dict[f'{mode} targets pass all fileters']/d.shape[0]:.2f}%)")


#### Cas9
2632337 (72%) targets hit a CDS
targets pass all fileters: 379147 (10.35%)
#### Cpf1
977085 (46%) targets hit a CDS
targets pass all fileters: 265419 (12.48%)


In [19]:
results_dict

OrderedDict([('total genes in gtf', 14626),
             ('number of protein colding genes', 13601),
             ('total coding length [bp]', 23255031),
             ('overlapping CDSs with different gene ids', 37),
             ('Cas9 targets with unique location', 3665000),
             ('Cas9 targets pass MM flt', 858176),
             ('Cas9 targets pass GC flt', 3016380),
             ('Cas9 targets pass Eff flt', 1946078),
             ('Cpf1 targets with unique location', 2126127),
             ('Cpf1 targets pass MM flt', 1695992),
             ('Cpf1 targets pass GC flt', 1249707),
             ('Cpf1 targets pass Eff flt', 716463),
             ('Cas9 targets hit CDS', 2632337),
             ('Cas9 targets pass all fileters', 379147),
             ('Cpf1 targets hit CDS', 977085),
             ('Cpf1 targets pass all fileters', 265419)])

In [20]:
# display_all(targets['Cpf1'].iloc[40:100,:])

## Filtering based on variant frequency from VCF (@TCC Modified to save all targets, not just potential)

## search vcf for target sites

In [21]:
### Open the callset (zarr verison of the vcf) and get the list of CHROMS
callset = zarr.open_group(ZVCF, mode='r')
if CHROMS is None: # use all chroms?
    CHROMS = list(callset.keys())

In [22]:
### get the (optional) subset of samples to use

if SAMPLE_LIST_FN is None: # use all samples
    meta = pd.DataFrame(list(callset.values())[0]['samples'][:])
    meta['callset_idx'] = range(meta.shape[0])
else:
    # load the sample list and make meta dataframe
    if isinstance(SAMPLE_LIST_FN, str): # assume a simple list file
        meta = pd.read_csv(SAMPLE_LIST_FN, comment='#', header=None, index_col=0, skiprows=1)
    else: # parse ag1000g samples.meta.txt with SAMPLE_LIST_FN really being [M|S,filename]
        meta = pd.read_csv(SAMPLE_LIST_FN[1], delimiter='\t', comment='#', index_col=0)
        meta = meta[meta['m_s']==SAMPLE_LIST_FN[0]]
    all_callset_samples = list(list(callset.values())[0]['samples'])
    meta['callset_idx'] = [all_callset_samples.index(x) for x in meta.index]
meta.index.name = 'sample'

# Optionally subsample to check sample size effects
if SUBSAMPLE_N:
    meta = meta.sample(n=SUBSAMPLE_N)

# ensure it is sorted by callset_idx (otherwise makes later steps terribly inefficient)
meta = meta.sort_values('callset_idx')

print("number of samples",meta.shape[0])
results_dict['number of samples'] = meta.shape[0]
# meta

number of samples 89


In [23]:
### get variant positiosn from the callset (zvcf)
ac_dict = OrderedDict()
refAF_dict = OrderedDict()
vpos_dict = OrderedDict()
results_dict['vcf variants'] = 0
results_dict['variants'] = 0

for chrom in CHROMS:
    print("chrom:", chrom)
    
    ## Get the positions of variants on this chrom (possible subsetting samples)
    pos = allel.SortedIndex(callset[chrom]['variants/POS'])
    g = allel.GenotypeDaskArray(callset[chrom]['calldata/GT']).subset(None, meta['callset_idx'].values)
    n_variants_in = g.shape[0]
    
    # ensure number of samples is same
    assert meta.shape[0] == g.shape[1]
    
    # get major allele frequencies
    ac = g.count_alleles().compute()
    # only variant alleles matter, so only take those
    flt_var = ac.is_variant()
    print('variant filter passes:', flt_var.sum())
    
    if IGNORE_ONE_CALL_VARIANTS:
        flt_one_call = ac[:,1:].sum(axis=1)>1
        print('ignore-one-call variant filter passes:', flt_one_call.sum())
        flt_var = (flt_var & flt_one_call)
    
    ac = ac.compress(flt_var, axis=0)
    n_variants_var = ac.shape[0]

    ac_dict[chrom] = ac
    refAF_dict[chrom] = ac.to_frequencies(fill=1)[:,0] # ref should be allele index 0
    vpos_dict[chrom] = pos.compress(flt_var, axis=0)[:] # `[:]` to loads into memory
    
    print("variants : {} of {} = {:.3f}%".format(n_variants_var, n_variants_in,
                                                 100*n_variants_var/n_variants_in))
    results_dict['vcf variants'] += n_variants_in
    results_dict['variants'] += n_variants_var
        
print('total variants :', results_dict['variants'])
# free up memory
del pos
del g
del ac

chrom: 1
variant filter passes: 37857469
variants : 37857469 of 42125744 = 89.868%
chrom: 2
variant filter passes: 67942773
variants : 67942773 of 74616963 = 91.055%
chrom: 3
variant filter passes: 56085822
variants : 56085822 of 61792617 = 90.765%
total variants : 161886064


In [24]:
%%time
##### Nucleotide diveristiy of coding regions calcluation #####
cds_grp_nuc_diversity = OrderedDict()
cds_grp_length = OrderedDict()
for chrom in CHROMS:

    gb = cds[cds['seqid']==chrom].groupby('overlap_group_ignoring_gene', dropna=False)
    starts = gb['start'].min().values
    ends = gb['end'].max().values

    cds_grp_nuc_diversity[chrom] = np.zeros(starts.shape)
    cds_grp_length[chrom] = np.zeros(starts.shape)

    for i in range(len(starts)):
        # print(starts[i], ends[i])
        cds_grp_length[chrom][i] = ends[i]-starts[i]+1
        try:
            cds_grp_nuc_diversity[chrom][i] = allel.sequence_diversity(vpos_dict[chrom], ac_dict[chrom], start=starts[i], stop=ends[i])
        except KeyError as e:
            assert starts[i] < ends[i] 
            # print("Warning: no variants in pos range",e)
            cds_grp_nuc_diversity[chrom][i] = 0
    # if i>100: break


s = 0
n = 0
for chrom in CHROMS:
    print(f"{chrom} pi: mean={np.mean(cds_grp_nuc_diversity[chrom]):.2g}  std={np.std(cds_grp_nuc_diversity[chrom]):.2g}", end="\t")
    print(f"overall_pi={(cds_grp_nuc_diversity[chrom]*cds_grp_length[chrom]).sum() / cds_grp_length[chrom].sum():.3g}")
    s += (cds_grp_nuc_diversity[chrom]*cds_grp_length[chrom]).sum()
    n += cds_grp_length[chrom].sum()
print("** nucleotide diversity in coding regions:", s/n)
results_dict['nucleotide diversity in coding regions'] = s/n

# @TCC: here's a one-liner to calculate total CDS pi 
# (np.concatenate(list(cds_grp_nuc_diversity.values())) * np.concatenate(list(cds_grp_length.values()))).sum() / np.concatenate(list(cds_grp_length.values())).sum()

1 pi: mean=0.013  std=0.012	overall_pi=0.0153
2 pi: mean=0.011  std=0.01	overall_pi=0.0125
3 pi: mean=0.011  std=0.01	overall_pi=0.0124
** nucleotide diversity in coding regions: 0.013172444056535816
CPU times: user 4.27 s, sys: 4.21 ms, total: 4.27 s
Wall time: 4.29 s


In [25]:
# fig, ax = mpl.pyplot.subplots(1,1, facecolor='white')
# cnts,bins,patches = ax.hist(np.concatenate(list(cds_grp_nuc_diversity.values())), bins=500)

In [26]:
results_dict

OrderedDict([('total genes in gtf', 14626),
             ('number of protein colding genes', 13601),
             ('total coding length [bp]', 23255031),
             ('overlapping CDSs with different gene ids', 37),
             ('Cas9 targets with unique location', 3665000),
             ('Cas9 targets pass MM flt', 858176),
             ('Cas9 targets pass GC flt', 3016380),
             ('Cas9 targets pass Eff flt', 1946078),
             ('Cpf1 targets with unique location', 2126127),
             ('Cpf1 targets pass MM flt', 1695992),
             ('Cpf1 targets pass GC flt', 1249707),
             ('Cpf1 targets pass Eff flt', 716463),
             ('Cas9 targets hit CDS', 2632337),
             ('Cas9 targets pass all fileters', 379147),
             ('Cpf1 targets hit CDS', 977085),
             ('Cpf1 targets pass all fileters', 265419),
             ('number of samples', 89),
             ('vcf variants', 178535324),
             ('variants', 161886064),
             ('nucle

In [27]:
## Overlap between targets and variants

for mode in ('Cas9', 'Cpf1'):
    # all target sequences for a given mode are the same length
    target_len = len(targets[mode].iloc[0]['Target sequence'])
    print(mode, target_len)

    tout = targets[mode].copy(deep=True)

    tout['vpos'] = np.nan # position of each variant
    tout['vpos'] = tout['vpos'].astype('object')
    tout['nv'] = np.nan # number of variants
    tout['refAF'] = np.nan # frequency of reference allele of each variant
    tout['refAF'] = tout['refAF'].astype('object')
    tout['p_ref'] = np.nan # probability of ref sequence for whole target

    for chrom in CHROMS:
        print("chrom:", chrom)
        vpos = vpos_dict[chrom]

        # target positions (target list is filtered above)
        tpos = targets[mode][targets[mode]['chrom']==chrom]['pos'].copy(deep=True)
        print("unique (location,gene) targets:", tpos.shape)
        
        # find target positions in the list of variant positions (using sorted insert positions)
        p1 = np.searchsorted(vpos, tpos.values, side='left')
        p2 = np.searchsorted(vpos, tpos.values+target_len, side='left')

        # list of actual variant positions in each target
        tout.loc[tpos.index,'vpos'] = np.array([vpos[_p1:_p2].values for _p1,_p2 in zip(p1,p2)], dtype=object)
        # count of variants in each target
        tout.loc[tpos.index,'nv'] = (p2-p1)
        # refAF of each variant in each target
        tout.loc[tpos.index,'refAF'] = np.array([refAF_dict[chrom][_p1:_p2] for _p1,_p2 in zip(p1,p2)], dtype=object)
        # the probability a target locus will be entirely reference (perfect match)
        # assumes each variant hitting that target is independent
        tout.loc[tpos.index,'p_ref'] = tout.loc[tpos.index,'refAF'].apply(lambda x: np.prod(x, initial=1))

    # make sure everything got filled in
    assert tout[tout['nv'].isna()].empty
    #
    targets[mode] = tout

Cas9 23
chrom: 1
unique (location,gene) targets: (928704,)
chrom: 2
unique (location,gene) targets: (1435679,)
chrom: 3
unique (location,gene) targets: (1300617,)
Cpf1 28
chrom: 1
unique (location,gene) targets: (483723,)
chrom: 2
unique (location,gene) targets: (883805,)
chrom: 3
unique (location,gene) targets: (758599,)


In [28]:
# free up memory by discarding the ac_dict, refAF_dict, vpos_dict  
del ac_dict
del refAF_dict
del vpos_dict

In [29]:
results_dict

OrderedDict([('total genes in gtf', 14626),
             ('number of protein colding genes', 13601),
             ('total coding length [bp]', 23255031),
             ('overlapping CDSs with different gene ids', 37),
             ('Cas9 targets with unique location', 3665000),
             ('Cas9 targets pass MM flt', 858176),
             ('Cas9 targets pass GC flt', 3016380),
             ('Cas9 targets pass Eff flt', 1946078),
             ('Cpf1 targets with unique location', 2126127),
             ('Cpf1 targets pass MM flt', 1695992),
             ('Cpf1 targets pass GC flt', 1249707),
             ('Cpf1 targets pass Eff flt', 716463),
             ('Cas9 targets hit CDS', 2632337),
             ('Cas9 targets pass all fileters', 379147),
             ('Cpf1 targets hit CDS', 977085),
             ('Cpf1 targets pass all fileters', 265419),
             ('number of samples', 89),
             ('vcf variants', 178535324),
             ('variants', 161886064),
             ('nucle

In [30]:
display_all_cols(targets['Cpf1'].head())

,Rank,Target sequence,Genomic location,Strand,GC content (%),Self-complementarity,Efficiency,MM0,MM1,MM2,MM3,transcript_id,gene_id,chrom,pos,cutpos,cutneg,flag unique location,MM3_num,sumMM,flt MM,flt GC,flt Eff,idx,flt hitsCDS,vpos,nv,refAF,p_ref
0,138,TTTTTTGATGTTTCATGTAGATGGCTCG,1:31480,+,42,0,4,0,0,0,0,AAEL012102-RB,None,1,31480,31503,31507,True,0.0,0.0,True,True,False,0,False,"[31481, 31485, 31486, 31487, 31493, 31495]",6.0,"[0.9943820224719101, 0.9887640449438202, 0.971...",0.849814
1,134,TTTTTGATGTTTCATGTAGATGGCTCGC,1:31481,+,46,0,4,0,0,0,0,AAEL012102-RB,None,1,31481,31504,31508,True,0.0,0.0,True,True,False,1,False,"[31481, 31485, 31486, 31487, 31493, 31495]",6.0,"[0.9943820224719101, 0.9887640449438202, 0.971...",0.849814
2,128,TTTTGATGTTTCATGTAGATGGCTCGCC,1:31482,+,50,0,7,0,0,0,0,AAEL012102-RB,None,1,31482,31505,31509,True,0.0,0.0,True,True,False,2,False,"[31485, 31486, 31487, 31493, 31495]",5.0,"[0.9887640449438202, 0.9719101123595506, 0.904...",0.854615
3,81,TTTGATGTTTCATGTAGATGGCTCGCCA,1:31483,+,46,1,41,0,0,0,0,AAEL012102-RB,None,1,31483,31506,31510,True,0.0,0.0,True,True,False,3,False,"[31485, 31486, 31487, 31493, 31495]",5.0,"[0.9887640449438202, 0.9719101123595506, 0.904...",0.854615
4,17,TTTCATGTAGATGGCTCGCCATCTTAGA,1:31490,+,46,3,75,0,0,0,0,AAEL012102-RA,None,1,31490,31513,31517,True,0.0,0.0,True,True,True,4,False,"[31493, 31495]",2.0,"[0.9887640449438202, 0.9943820224719101]",0.983209


In [31]:
%%time
# Saving targets dataframe
with open(f'{SETNAME}_CDS_targets.pickle', 'wb') as fh:
    pickle.dump(targets, fh)
with open(f'{SETNAME}_CDS_results_dict.pickle', 'wb') as fh:
    pickle.dump(results_dict, fh)

CPU times: user 44.2 s, sys: 2.26 s, total: 46.4 s
Wall time: 48.4 s


In [32]:
%%time
# saving subset of potential targets (passing all filters except population variation)
# this is much smaller than all the unique targets, so speeds up the later analysis, figure generation, ect.
potential_targets = OrderedDict()

for mode in targets.keys():
    print('####', mode)
    d = targets[mode]
    potential_targets[mode] = d[(d['flt MM']==True) & (d['flt GC']==True) & (d['flt Eff']==True) & (d['flt hitsCDS']==True)].copy()

# Saving the results and potential_targets dataframe (smaller and used for analysis)
with open(f'{SETNAME}_potential_targets.pickle', 'wb') as fh:
    pickle.dump((potential_targets, results_dict), fh)

#### Cas9
#### Cpf1
CPU times: user 5.34 s, sys: 236 ms, total: 5.58 s
Wall time: 6.48 s
